In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
# Clear any logs from previous runs
!rm -rf ./logs/

In [3]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from tensorboard.plugins.hparams import api as hp

In [4]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

4431872/4422102 [==============================] - 0s 0us/step


In [5]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test =x_test.reshape(x_test.shape[0], 28, 28, 1)

In [6]:
NUM_UNIT_1 = hp.HParam('num1', hp.Discrete([32, 64, 128]))
NUM_UNIT_2 = hp.HParam('num2', hp.Discrete([32, 64, 128]))
NUM_UNIT_3 = hp.HParam('num3', hp.Discrete([32, 64, 128]))
NUM_UNIT_4 = hp.HParam('num4', hp.Discrete([32, 64, 128]))
METRIC_ACCURACY = 'accuracy'
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(hparams=[NUM_UNIT_1,NUM_UNIT_2,NUM_UNIT_3,NUM_UNIT_4], metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')])

In [7]:
def train_test_model(hparams):
  model = keras.Sequential([
    keras.layers.Conv2D(hparams[NUM_UNIT_1], kernel_size = (5,5), padding = 'same', input_shape = (28,28,1)),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('selu'),
    keras.layers.Conv2D(hparams[NUM_UNIT_2], kernel_size = (5,5), padding = 'same'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling2D(pool_size = (2, 2)),
    keras.layers.Dropout(0.30),
    keras.layers.Conv2D(hparams[NUM_UNIT_3], [3, 3], padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Conv2D(hparams[NUM_UNIT_4], [3, 3], padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling2D(pool_size = (2, 2)),
    keras.layers.Dropout(0.30),
    keras.layers.Flatten(),
    keras.layers.Dense(1024, activation ='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(512, activation ='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.30),
    keras.layers.Dense(10, activation ='softmax'), 
])
  model.compile(optimizer='Nadam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  model.fit(x_train, y_train, epochs=10) # Run with 1 epoch to speed things up for demo purposes
  _, accuracy = model.evaluate(x_test, y_test)
  return accuracy

In [8]:
Accuracy = []
combination = 0
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    global combination
    combination = combination + 1
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    Accuracy.append(['combination'+str(combination),list(hparams.values())[0],list(hparams.values())[1],list(hparams.values())[2],list(hparams.values())[3],accuracy])
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [9]:
session_num = 0
# for num_units in HP_NUM_UNITS.domain.values:
#  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
for num1 in NUM_UNIT_1.domain.values:
  for num2 in NUM_UNIT_2.domain.values:
    for num3 in NUM_UNIT_3.domain.values:
      for num4 in NUM_UNIT_4.domain.values:
        hparams = {NUM_UNIT_1: num1,NUM_UNIT_2: num2,NUM_UNIT_3: num3,NUM_UNIT_4: num4}
        run_name = str(num1)
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('logs/hparam_tuning/' + run_name, hparams)     
        session_num += 1

--- Starting trial: 32
{'num1': 32, 'num2': 32, 'num3': 32, 'num4': 32}
Epoch 1/10
1875/1875 [==============================] - 37s 10ms/step - loss: 0.5152 - accuracy: 0.8183
Epoch 2/10
1875/1875 [==============================] - 19s 10ms/step - loss: 0.3449 - accuracy: 0.8749
Epoch 3/10
1875/1875 [==============================] - 18s 10ms/step - loss: 0.3092 - accuracy: 0.8876
Epoch 4/10
1875/1875 [==============================] - 18s 10ms/step - loss: 0.2831 - accuracy: 0.8962
Epoch 5/10
1875/1875 [==============================] - 18s 9ms/step - loss: 0.2610 - accuracy: 0.9047
Epoch 6/10
1875/1875 [==============================] - 19s 10ms/step - loss: 0.2469 - accuracy: 0.9098
Epoch 7/10
1875/1875 [==============================] - 19s 10ms/step - loss: 0.2315 - accuracy: 0.9151
Epoch 8/10
1875/1875 [==============================] - 18s 10ms/step - loss: 0.2175 - accuracy: 0.9199
Epoch 9/10
1875/1875 [==============================] - 20s 10ms/step - loss: 0.2082 - accuracy: 

In [10]:
df = pd.DataFrame(Accuracy,columns=('combination_no:','Num_of_units_1','Num_of_units_2','Num_of_units_3','Num_of_units_4', 'Accuracy'))
df
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(df)
column = df["Accuracy"]
max_value = column.idxmax()

,combination_no:,Num_of_units_1,Num_of_units_2,Num_of_units_3,Num_of_units_4,Accuracy
0,combination1,32,32,32,32,0.9258
1,combination2,32,32,32,64,0.9200
2,combination3,32,32,32,128,0.9272
3,combination4,32,32,64,32,0.9328
4,combination5,32,32,64,64,0.9280
5,combination6,32,32,64,128,0.9304
6,combination7,32,32,128,32,0.9301
7,combination8,32,32,128,64,0.9309
8,combination9,32,32,128,128,0.9309
9,combination10,32,64,32,32,0.9251


In [11]:
print("The best model combination is displayed as follows:\n")
df.iloc[max_value,:] 

The best model combination is displayed as follows:



combination_no:    combination51
Num_of_units_1                64
Num_of_units_2               128
Num_of_units_3                64
Num_of_units_4               128
Accuracy                  0.9389
Name: 50, dtype: object